In [1]:
import tensorflow as tf

In [2]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import imblearn
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_df = pd.read_csv('econet dataset/train.csv')

In [4]:
train_df['Ob'] = pd.to_datetime(train_df['Ob'])

In [5]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
label_encoder = LabelEncoder()
features = ['Station', 'Ob', 'value', 'measure', 'R_flag', 'I_flag', 'Z_flag', 'B_flag']
temp_df = train_df.loc[:]
label_encoder = LabelEncoder()
temp_df['Station'] = label_encoder.fit_transform(train_df['Station'])
temp_df['measure'] = label_encoder.fit_transform(train_df['measure'])
temp_df['Ob'] = train_df['Ob'].values.astype(float)

x = temp_df.loc[:, features].values
y = temp_df.loc[:,['target']].values
x = StandardScaler().fit_transform(x)

In [6]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components = 0.95)
# principalComponents = pca.fit_transform(x)
# p_df = pd.DataFrame(principalComponents)

In [7]:
finalDf = pd.concat([pd.DataFrame(x), temp_df[['target']]], axis = 1)
print(finalDf.head())

         0         1         2         3         4         5         6  \
0 -1.57635 -1.619118 -0.251460  1.351229  1.280241 -0.400139  1.233982   
1 -1.57635 -1.617488 -0.249337  1.351229  1.280241 -0.400139  1.233982   
2 -1.57635 -1.617080 -0.250398  1.351229  1.280241 -0.400139  1.233982   
3 -1.57635 -1.616265 -0.253937  1.351229  1.280241 -0.400139  1.233982   
4 -1.57635 -1.178027 -0.256768  1.351229  1.280241 -0.400139  1.233982   

          7  target  
0  0.431695   False  
1  0.431695   False  
2  0.431695   False  
3  0.431695   False  
4  0.431695   False  


In [8]:
y = finalDf['target']
X = finalDf[[0, 1, 2, 3, 4, 5, 6, 7]]
print(type(y), type(X))

<class 'pandas.core.series.Series'> <class 'pandas.core.frame.DataFrame'>


In [ ]:
# finalDf.to_csv('finalDF.csv', index = True)
# finalDf = pd.read_csv('finalDF.csv')

In [9]:
print(X.shape, y.shape)

(6593274, 8) (6593274,)


In [32]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [33]:
from imblearn.over_sampling import SMOTE

under = SMOTE(random_state = 42)

X_train, y_train = under.fit_resample(X_train, y_train)
print(X_train.shape, y_train.shape)

(8901692, 8) (8901692,)


In [34]:
pd.DataFrame(y_train).value_counts()

target
False     4450846
True      4450846
dtype: int64

In [35]:
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

In [36]:
print(y_train, y_test)

[[False]
 [False]
 [False]
 ...
 [ True]
 [ True]
 [ True]] [[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


In [37]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc = enc.fit(y_train)
y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

In [38]:
print(X_train.shape, y_train.shape)

(8901692, 8) (8901692, 2)


In [39]:
print(y_train)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [40]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1, min_delta = 0.001)

In [42]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam, SGD

input_shape = (X_train.shape[1])
output_dim = 2

ann=Sequential()
ann.add(Dense(units=input_shape, activation="relu"))
ann.add(Dropout(0.1))
ann.add(Dense(units=16,activation="relu"))
ann.add(Dense(units=16,activation="relu"))
ann.add(Dropout(0.1))
ann.add(Dense(units=2,activation="sigmoid"))
ann.compile(optimizer="adam",loss="binary_crossentropy")

In [ ]:
hist = ann.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=3)

Epoch 1/3
197327/222543 [=========================>....] - ETA: 3:44 - loss: 0.1744

In [ ]:
import pickle
filename = 'ann_model.pkl'
with open(filename, 'wb') as f:
    pickle.dump(hist,  f)

In [ ]:
y_pred = ann.predict(X_test)

In [ ]:
print(y_pred)
print(y_pred.shape)

In [ ]:
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)

In [ ]:
print(y_pred)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
test_df = pd.read_csv('econet dataset/test.csv')

In [ ]:
test_df['Ob'] = pd.to_datetime(test_df['Ob'])

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

features = ['Station', 'Ob', 'value', 'measure', 'R_flag', 'I_flag', 'Z_flag', 'B_flag']
temp_df = test_df.loc[:]

label_encoder = LabelEncoder()
temp_df['Station'] = label_encoder.fit_transform(test_df['Station'])
temp_df['measure'] = label_encoder.fit_transform(test_df['measure'])
temp_df['Ob'] = test_df['Ob'].values.astype(float)

x = temp_df.loc[:, features].values
# x = StandardScaler().fit_transform(x)

In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components = 0.95)
# principalComponents = pca.fit_transform(x)
# p_df = pd.DataFrame(principalComponents)

In [ ]:
finalDf = x

In [ ]:
print(finalDf)

In [ ]:
y_preds_test = ann.predict(finalDf)

In [ ]:
print(y_preds_test.shape)

In [ ]:
pd.DataFrame(y_preds_test[:,1], columns=['target']).to_csv('predictions.csv', index=False)